In [3]:
import numpy as np
import pandas as pd

In [4]:
data  = pd.read_json("/Data2/Arun-UAV/NLP/vision_halu/evidence_head_train_datasets/finecops_ref/expression_all_train_set.json")

In [6]:
d_t = data.T

In [212]:
d_t.head(4)

,id,image_id,objects_id,tuple,tuple_type,level,spatial,attribute,group_id,instance_id,...,positive_id,negative_level,negative_type,shuffle_index,ori_type,replace_index,origin_name,replace_name,ori_attribute,neg_attr_cate
1,1,2386621,[681258],[[bowl]],0_hop,1,to the center,[[full]],-1,681258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2386621,"[681258, 681255, 681254]","[[bowl, next to, plate], [plate, of, meal]]",2_hop,1,to the center,"[[full], [white, full], []]",-1,681258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2386621,"[681258, 681255, 681257]","[[bowl, next to, plate], [bowl, of, dish]]",and,1,to the center,"[[full], [white, full], [cream colored]]",-1,681258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2386621,[681256],[[spoon]],0_hop,1,to the left,"[[large, metal, silver]]",-1,681256,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
pos_data = d_t[d_t["positive_id"].isna()]
neg_data = d_t[~d_t["positive_id"].isna()]
neg_data = neg_data[neg_data["neg_attr_cate"].isna()]

In [157]:
neg_df = neg_data[(neg_data["expression"].apply(lambda x: len(x.split(" "))) > 15)]
pos_df = pos_data[(pos_data["expression"].apply(lambda x: len(x.split(" "))) > 15)]
neg_df = neg_df.sample(20000)
pos_df = pos_df.sample(10000)

In [144]:
neg_df_obj = neg_df[neg_df["negative_type"] == "object"]
neg_df_n_obj = neg_df[neg_df["negative_type"] != "object"]

In [167]:
neg_df_obj.head(2)

,id,image_id,objects_id,tuple,tuple_type,level,spatial,attribute,group_id,instance_id,...,positive_id,negative_level,negative_type,shuffle_index,ori_type,replace_index,origin_name,replace_name,ori_attribute,neg_attr_cate
254670,254670,2342475,"[3090622, 2780622, 3090622]","[[train, same color, fence], [fence, to the ri...",same_attr_two_hop,1,NaN,black,NaN,NaN,...,186166,NaN,object,NaN,NaN,2,train,motorcycle,NaN,NaN
240316,240316,2403098,"[383655, 383658, 383653]","[[cutting board, same materials, table], [tabl...",same_attr_two_hop,1,NaN,wood,NaN,NaN,...,185992,NaN,object,NaN,NaN,2,wine glass,coffee mug,NaN,NaN


In [171]:
total_df = pd.concat([pos_df, neg_df_n_obj])

In [172]:
total_df.shape

(24897, 23)

In [176]:
from tqdm import tqdm

all_res = []
for inx, row in tqdm(neg_df_obj.iterrows()):
    image_id = row['image_id']
    vg_objects = pd.DataFrame(annot[annot['image_id'] == image_id]["objects"].iloc[0])
    vg_objects["object_id"] = vg_objects["object_id"].apply(lambda x: str(int(x)))
    object_ids = row["objects_id"]
    object_info = vg_objects[vg_objects["object_id"].isin(object_ids)][["object_id", "names", "h", "w", "y", "x"]].to_dict(orient="records")
    answer = row["expression"]
    all_res.append({"image_id": image_id, "answer": answer, "objects_info": object_info, "replacement_info": {row["origin_name"]: row["replace_name"]}})

0it [00:00, ?it/s]

5103it [01:09, 72.99it/s]


In [174]:
process_df = pd.DataFrame(all_res)

In [178]:
process_neg_df = pd.DataFrame(all_res)

In [215]:
all_fil_info = []
for i, row in process_neg_df.iterrows():
    
    fil_info = []
    for i in row["objects_info"]:
        if i["names"] == list(row["replacement_info"].keys()):
            continue
        else:
            fil_info.append(i)

    fil_info.append({'object_id': '-1',
    'names': list(row["replacement_info"].values()),
    'h': 0,
    'w': 0,
    'y': 0,
    'x': 0})
    all_fil_info.append(str(fil_info))

In [217]:
process_neg_df["objects_info"] = all_fil_info

In [219]:
process_neg_df = process_neg_df.drop("replacement_info", axis=1)

In [220]:
process_neg_df.head(2)

,image_id,answer,objects_info
0,2342475,The train sharing the same color as the fence ...,"[{'object_id': '2780622', 'names': ['fence'], ..."
1,2403098,The cutting board of the same materials as the...,"[{'object_id': '383655', 'names': ['cutting bo..."


In [222]:
process_df.head(2)

,image_id,answer,objects_info
0,2373626,"The dish, positioned to the right of the gray ...","[{'object_id': '3725798', 'names': ['dish'], '..."
1,285719,"On the left side of the metal and black fence,...","[{'object_id': '4401457', 'names': ['bike'], '..."


In [223]:
total_process_df = pd.concat([process_df, process_neg_df])

# visdual genome 

In [ ]:
annot = pd.read_json("/Data2/Arun-UAV/NLP/vision_halu/visual_genome/annotations/objects.json")
relations = pd.read_json("/Data2/Arun-UAV/NLP/vision_halu/visual_genome/annotations/relationships.json")
attributes = pd.read_json("/Data2/Arun-UAV/NLP/vision_halu/visual_genome/annotations/attributes.json")

In [28]:
annot.head()

,image_id,objects,image_url,merged_object_ids
0,1,"[{'synsets': ['tree.n.01'], 'h': 557, 'object_...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
1,2,"[{'synsets': ['road.n.01'], 'h': 254, 'object_...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
2,3,"[{'synsets': ['booth.n.02'], 'h': 389, 'object...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
3,4,"[{'synsets': ['floor.n.01'], 'h': 168, 'object...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
4,5,"[{'synsets': ['room.n.01'], 'h': 599, 'object_...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN


In [34]:
pd.DataFrame(annot[annot["image_id"] == 2404820]["objects"].iloc[0])

,synsets,h,object_id,merged_object_ids,names,w,y,x
0,[background.n.02],48,375770,[],[background],496,1,4
1,[bottom.n.01],34,375773,[],[bottom],65,254,3
2,[bread.n.01],243,375755,[],[bread],251,65,248
3,[rotter.n.01],63,375764,[],[crumbs],71,197,408
4,[crust.n.02],76,375760,[],[crust],98,69,345
5,[edge.n.01],34,375763,[],[edge],162,291,337
6,[merchandise.n.01],181,375759,[],[food],247,193,63
7,[glass.n.01],77,375758,[],[glass],130,2,369
8,[line.n.01],53,375765,[],[lines],62,145,369
9,[liquid.n.01],43,375757,[],[liquid],78,25,385


In [45]:
pd.DataFrame(relations[relations["image_id"] == 2404820]["relationships"].iloc[0])["object"].iloc[20]

{'name': 'crust',
 'h': 137,
 'synsets': ['crust.n.02'],
 'object_id': 375760,
 'w': 198,
 'y': 69,
 'x': 299}

In [48]:
qa = pd.read_json("/Data2/Arun-UAV/NLP/vision_halu/visual_genome/annotations/question_answers.json")

In [54]:
df = pd.DataFrame(qa["qas"].iloc[2])

In [ ]:
visualgenome + annotation

In [ ]:
caption, instruct, vqa
